In [ ]:
import os

os.environ["WIKIDATA_CACHE_FILE"] = "../.wikidata-cache.json"

In [ ]:
# https://www.bp.com/en/global/corporate/energy-economics/statistical-review-of-world-energy.html

from shifter_pandas.bp import UNITS_ENERGY, BPDatasource

shifter_ds = BPDatasource("../data/bp-stats-review-2021-all-data.xlsx")

In [ ]:
for type_ in shifter_ds.metadata():
    if type_["supported"]:
        print(f"{type_['label']} [{type_['unit']['iso']}]")

In [ ]:
shifter_ds.oil_units_conversion

In [ ]:
shifter_ds.oil_products_units_conversion

In [ ]:
shifter_ds.gaz_units_conversion

In [ ]:
r = {}
for e in shifter_ds.metadata():
    if type_["supported"]:
        r[e["unit"]["original"]] = (
            f"[{e['unit']['iso']}]{e['unit']['iso_postfix']} => {e['unit']['iso_factor']}"
        )
r

In [ ]:
{e["unit"]["iso"] for e in shifter_ds.metadata() if e["supported"]}

In [ ]:
shifter_ds.to_iso_unit

In [ ]:
df = shifter_ds.datasource_non_fossil_electricity_to_primary_energy_factor(1995)
df

In [ ]:
import plotly.express as px

fig = px.line(df, x="Year", y="Factor")
fig.show()

In [ ]:
df = shifter_ds.datasource(
    regions_filter=["Total World"],
    types_filter=["Oil Production - Barrels", "Oil Production - Tonnes", "Oil Production - Crude Conds"],
)
for col in ["TypeUnit", "Region"]:
    if col in df:
        del df[col]
df

In [ ]:
df_vol = df.query('Type == "Oil Production - Barrels"')
df_vol.rename(columns={"Value": "OilProductionVolume"}, inplace=True)
del df_vol["Type"]
del df_vol["Unit"]
df_mass = df.query('Type == "Oil Production - Tonnes"')
df_mass.rename(columns={"Value": "OilProductionMass"}, inplace=True)
del df_mass["Type"]
del df_mass["Unit"]
df_energy = df.query('Type == "Oil Production - Crude Conds"')
df_energy.rename(columns={"Value": "OilProductionEnergy"}, inplace=True)
del df_energy["Type"]
del df_energy["Unit"]
df_energy

In [ ]:
import pandas as pd

df_merged = pd.merge(
    df_vol,
    df_mass,
    how="inner",
    on=["Year"],
)
df_merged = pd.merge(
    df_merged,
    df_energy,
    how="inner",
    on=["Year"],
)
df_merged

In [ ]:
df_final = df_merged.assign(
    OilProductionEnergyVolume=df_merged.OilProductionEnergy / df_merged.OilProductionVolume
)
df_final = df_final.assign(
    OilProductionEnergyMass=df_merged.OilProductionEnergy / df_merged.OilProductionMass
)
df_final = df_final.assign(
    OilProductionMassVolume=df_merged.OilProductionMass / df_merged.OilProductionVolume
)
df_final

In [ ]:
import plotly.express as px

fig = px.line(df_final, x="Year", y="OilProductionEnergyVolume")
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(df_final, x="Year", y="OilProductionEnergyMass")
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(df_final, x="Year", y="OilProductionMassVolume")
fig.show()